# Clock Jitter

This notebook demonstates the functional of the clock jitter transform. The transform models the sampling phase having some error, represented by a Gaussian random variable whose mean is zero and whose variance is according to the PPM error parameter. The effect will increase or decrease the sampling phase randomly by a fractional sample. Over long sequences this will produce a handful of more or less samples than were input.

In [ ]:
import torchsig.transforms.functional as F
from torchsig.utils.defaults import default_dataset

import numpy as np
import scipy as sp

%matplotlib inline
import matplotlib.pyplot as plt

A tone signal is generated to test the clock jitter transform. The error is defined in parts-per-million (PPM). A larger PPM represents a larger frequency error on average. Using 0 PPM will result in no sampling phase error.

In [ ]:
# test functional
rng = np.random.default_rng(42)

N = 10000
sample_rate = 10e6
# center_frequency = 0
center_frequency = sample_rate / N
n = np.arange(0, N)
t = n / sample_rate
tone_bb_data = np.exp(2j * np.pi * center_frequency * t)

jitter_ppm = 1

data_out = F.clock_jitter(data=tone_bb_data, jitter_ppm=jitter_ppm, rng=rng)

Time-domain plots show how the clock jitter effects the complex sinusoid input. For small PPM errors the effect will may not be visible in the time domain.

In [ ]:
fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2, 1, 1)
ax.plot(t, np.real(tone_bb_data), label="Real, Input Tone")
ax.plot(t, np.imag(tone_bb_data), label="Imag, Input Tone")
ylim = np.max(np.abs(tone_bb_data)) * 1.1
ax.set_ylim([-ylim, ylim])
ax.set_xlim([t[0], t[-1]])
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude")
ax.grid()
ax.legend(loc="upper right")

ax = fig.add_subplot(2, 1, 2)
ax.plot(t, np.real(data_out), label="Real, Tone with Clock Jitter")
ax.plot(t, np.imag(data_out), label="Imag, Tone with Clock Jitter")
ax.set_ylim([-ylim, ylim])
ax.set_xlim([t[0], t[-1]])
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude")
ax.grid()
ax.legend(loc="upper right")

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 10, scale: float = 1.0):
    """Generate a scaled, high SNR baseband QPSK Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 10.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["qpsk"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        start_in_samples=0,
        seed=67,
    )
    signal = dataset.signal_generators[0]()
    signal.data = signal.data * scale
    return signal

A QPSK signal is used to demonstrate the jitter. A larger jitter PPM value is used to make the effect more obvious.

In [ ]:
# test data
N = 128
qpsk_signal = generate_qpsk_signal(num_iq_samples=N)
qpsk_data = qpsk_signal.data

# phase noise
impaired_qpsk_data = F.clock_jitter(data=qpsk_data, jitter_ppm=jitter_ppm * 100)

n = np.arange(0, len(qpsk_data.data))
t = n / sample_rate

fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2, 1, 1)
ax.plot(t, np.real(qpsk_data), label="Real, Input QPSK")
ax.plot(t, np.real(impaired_qpsk_data), label="Real, QPSK with Jitter")
ylim = np.max(np.abs(qpsk_data)) * 1.1
# ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0], t[-1]])
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude")
ax.grid()
ax.legend(loc="upper right")

ax = fig.add_subplot(2, 1, 2)
ax.plot(t, np.imag(qpsk_data), label="Imag, Input QPSK")
ax.plot(t, np.imag(impaired_qpsk_data), label="Imag, QPSK with Jitter")
ax.set_ylim([-ylim, ylim])
ax.set_xlim([t[0], t[-1]])
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude")
ax.grid()
ax.legend(loc="upper right")